# Pipelines

Let's now take a closer look at pipelines. We will be using the **deep**doctection analyzer as an example. 

A pipeline is built as a sequence of tasks. These tasks are called pipeline components, pipeline backbones or services.

![pipelines](./pics/dd_overview_pipeline.png)

Once a pipeline is defined, images or documents can be processed. These are either pure image files (like JPG, PNG, TIFF) or PDF files. PDF files are read and processed page by page. Each PDF page is converted into a numpy array under the hood. 

We do not want to go into detail about the data structure at this point. If you want more information, please refer to the [data structure notebook](./Data_structure.ipynb).

In [1]:
import deepdoctection as dd

In [ ]:
analyzer = dd.get_dd_analyzer()

Let's take a closer look at the **deep**doctection analyzer. 

![pipeline](./pics/dd_pipeline.png)

The default construction consists of several pipeline components: 

- Layout analysis (object detection)
- Cell analysis in table regions (object detection)
- Row and column analysis in table regions (object detection)
- Table segmentation 
- Table segmentation refinement
- OCR with Tesseract
- Assignment of words to layout segments
- Reading order of words within layout segments and reading order of layout segments.

Therefore in total, three object detectors and one OCR are loaded.

## Configuration

![config](./pics/conf_dd_one_yaml.png)

We see while initializing a configuration in the logs of the analyzer. The configuration is saved in a `.yaml` file. You can find this file in the .cache dir of **deep**doctection.

You can use the `.yaml` file to replace one model with e.g. a model trained on your own data. The tutorial [**Analyzer_Configuration.ipynb**](./Analyzer_Configuration.ipynb) will show you where you need to pay attention when changing the `.yaml` file.

In [this tutorial](./Running_pre_trained_models_from_third_party_libraries.ipynb) we will show you how to add a model to the `ModelCatalog` and change the model in the `.yaml` file so that you can use model from third party libraries, that run layout detection models with Detectron2.

## Pipeline components

Having a pipeline, you can list the components with `get_pipeline_info()`.

In [3]:
analyzer.get_pipeline_info()

{0: 'image_weights_layout_d2_model_0829999_layout_inf_only.pt',
 1: 'sub_image_weights_item_d2_model_1639999_item_inf_only.pt',
 2: 'sub_image_weights_cell_d2_model_1849999_cell_inf_only.pt',
 3: 'table_segment',
 4: 'table_segment_refine',
 5: 'text_extract_tesseract',
 6: 'matching',
 7: 'text_order'}

In [4]:
analyzer.get_pipeline_info(position=3)

'table_segment'

If you do not want to process any text extraction you can set `config_overwrite=["USE_OCR=False"]` which gives you a shorter pipeline with fewer backbones.

In [ ]:
analyzer = dd.get_dd_analyzer(config_overwrite=["USE_OCR=False"])

The `config_overwrite` option allows to overwrite every argument specified in the `.yaml` file. E.g. you can overwrite `SEGMENTATION.ASSIGNMENT_RULE` simply by `config_overwrite=["SEGMENTATION.ASSIGNMENT_RULE=True"]`.

In [6]:
analyzer.get_pipeline_info()

{0: 'image_weights_layout_d2_model_0829999_layout_inf_only.pt',
 1: 'sub_image_weights_item_d2_model_1639999_item_inf_only.pt',
 2: 'sub_image_weights_cell_d2_model_1849999_cell_inf_only.pt',
 3: 'table_segment',
 4: 'table_segment_refine'}

You have access to pipeline components via `pipe_component_list`.

In [7]:
analyzer.pipe_component_list

## Layout detection models

The `ImageLayoutService` is responsible to detect the coarse layout structure over a full image. It has an object
detector, which can be either a Tensorpack or a Detectron2 model.

In [8]:
image_layout_service = analyzer.pipe_component_list[0]

In [9]:
image_layout_service.predictor

You can get a list of all categories that a model is able to detect. Moreover, you will find a unique description of each model in your pipeline.

In [10]:
image_layout_service.predictor.possible_categories()

[<LayoutType.text>,
 <LayoutType.title>,
 <LayoutType.list>,
 <LayoutType.table>,
 <LayoutType.figure>]

In [11]:
image_layout_service.predictor.name

'weights_layout_d2_model_0829999_layout_inf_only.pt'

In [12]:
cell_service = analyzer.pipe_component_list[1]

In [13]:
cell_service.predictor.possible_categories()

[<LayoutType.row>, <LayoutType.column>]

In [14]:
cell_service.predictor.name

'weights_item_d2_model_1639999_item_inf_only.pt'

## OCR and matching words to layout segments

Let's re-load the analyzer again, now with OCR.

In [ ]:
analyzer = dd.get_dd_analyzer()

The matching services maps words to layout segments by overlapping rules.  In order to do so, we need to specify what layout segments we want to consider.

```yaml
WORD_MATCHING:
  PARENTAL_CATEGORIES:
    - text
    - title
    - list
    - cell
    - column_header
    - projected_row_header
    - spanning
    - row_header
  RULE:  ioa
  THRESHOLD:  0.6
```

In this situation we do not consider `figure` as valid layout section and neglect any overlapping of a word with a `figure` segment. Of course, this can be changed by adding `figure` to the list of `parent_categories` or in `WORD_MATCHING.PARENTAL_CATEGORIES` in the `.yaml` file.

What is going to happen with so called orphan words, e.g. words with no overlapping with any layout segment? They simply have no anchor and will be ignored unless we force to process them as well. We will come to this point later. 

In [16]:
match_service = analyzer.pipe_component_list[6]

In [17]:
print(f"parent_categories: {match_service.parent_categories}, child_categories: {match_service.child_categories}")

parent_categories: ['text', 'title', 'list', 'cell', 'column_header', 'projected_row_header', 'spanning', 'row_header'], child_categories: word


There is a matching rule and a threshold to specifiy. We also need to choose whether we want to assign a word to 
multiple layout sections. When setting `max_parent_only=True` we assign the word to the layout section with the largest overlapping. Otherwise note, that the word might be considered twice. Changing `max_parent_only` from the `.yaml` is not provided.

In [18]:
print(f"matching_rule: {match_service.matching_rule} \n match_threshold: {match_service.threshold} \n max_parent_only: {match_service.max_parent_only}")

matching_rule: ioa 
 match_threshold: 0.6 
 max_parent_only: False


## Reading order

In the last step, words and layout segments must be arranged to create continuous text. This all takes place in 
the component `TextOrderService`.

Words that are assigned to layout segments are grouped into lines. Lines are read from top to bottom. 
Auxiliary columns are formed to sort the layout segments. These auxiliary columns are then grouped into contiguous blocks that span vertically across the page. Then the blocks are arranged so that adjacent columns in the contiguous blocks are read from left to right, and the contiguous blocks are read from top to bottom. 

![pipelines](./pics/dd_connected_blocks.png)


This order is, of course, completely arbitrary and will not result in the expected reading order for many layout compositions. 

An additional difficulty may be that the layout detection is not sufficiently precise and the algorithm returns a questionable reading order. This should always be kept in mind!

`TextOrderService` has four important parameters: `text_container`, `text_block_categories`, `floating_text_block_categories` and `include_residual_text_container`. 

`text_container` must contain the category that contains characters, e.g. `word`. 

`text_block_categories` contains all layout segments to which words have been added and which must be ordered.

`floating_text_block_categories` contains the text blocks to be included in the floating text. For example, it can be discussed whether tables should be included in the body text. In this configuration they are not included in the text. 

Let's get back to the orphan words: If we set `include_residual_text_container = False`, these words will not receive a `reading_order` and will be ignored in text output.

If, on the other hand, we set `include_residual_text_container = True`, they will be grouped and combined into lines and included to the text output. Thus no words are lost. This is an important configuration and you'll likely need to change it.

We refer to [this page](https://deepdoctection.readthedocs.io/en/latest/tutorials/layout_parsing_structure) for more detailed information about layout parsing and text ordering.

Let's have a look how the text order configs are reflected in the `.yaml`.

```yaml
TEXT_ORDERING:
  TEXT_BLOCK_CATEGORIES:
    - title
    - text
    - list
    - cell
    - column_header
    - projected_row_header
    - spanning
    - row_header
  FLOATING_TEXT_BLOCK_CATEGORIES:
    - title
    - text
    - list
  INCLUDE_RESIDUAL_TEXT_CONTAINER: False
  STARTING_POINT_TOLERANCE: 0.005
  BROKEN_LINE_TOLERANCE: 0.003
  HEIGHT_TOLERANCE: 2.0
  PARAGRAPH_BREAK: 0.035
```

In [21]:
text_order_service = analyzer.pipe_component_list[7]

In [22]:
print(f"text_container: {text_order_service.text_container} \n floating_text_block_categories: {text_order_service.floating_text_block_categories} \n text_block_categories: {text_order_service.text_block_categories} \n include_residual_text_container: {text_order_service.include_residual_text_container}")

text_container: word 
 floating_text_block_categories: [<LayoutType.title>, <LayoutType.text>, <LayoutType.list>] 
 text_block_categories: [<LayoutType.title>, <LayoutType.text>, <LayoutType.list>, <LayoutType.cell>, <CellType.column_header>, <CellType.projected_row_header>, <CellType.spanning>, <CellType.row_header>] 
 include_residual_text_container: False


## Output structure

There is a last step in a pipeline that prepares all information gathered from the different into a consumable class, the `Page` class. The `PageParsingService` is optional and should only be processed if you want to analyze the output.

For a deeper understanding of the connection between `Page` and `Image`, we refer to the [data structure notebook](./Data_structure.ipynb).

In [ ]:
df = analyzer.analyze(path="path/to/doc.pdf", output="image")  # output = "image" will skip PageParsingService. But default value is "page"

Note, that the `PageParsingService` is saved in a separate attribute and not part of `analyzer.pipe_component_list`. The `PageParsingService` shares some common parameters with the `TextOrderService`
and it is recommended to use the same configurations. 

In [23]:
page_parser = analyzer.page_parser

In [24]:
print(f"text_container: {page_parser.text_container} \n floating_text_block_categories: {page_parser.floating_text_block_categories} \n include_residual_text_container: {page_parser.include_residual_text_container}")

text_container: word 
 floating_text_block_categories: [<LayoutType.title>, <LayoutType.text>, <LayoutType.list>] 
 include_residual_text_container: False
